# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, fbeta_score
from sklearn.metrics import classification_report ,confusion_matrix
from scipy.stats import hmean
from scipy.stats.mstats import gmean
from sklearn.pipeline import FeatureUnion

import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Phongsapon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Phongsapon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse',engine)
X = df['message']
Y = df.iloc[:,4:]

In [6]:
#Checking for binary class
df['related'].value_counts()

1    20093
0     6122
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    
    #Converting Lower Case
    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower())
    
    #Tokenize word
    tokens = word_tokenize(text)
    
    #Normalization word with Stemming and Stop words
    normalizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normalize_tokens = [normalizer.stem(word) for word in tokens if word not in stop_words]
    
    return normalize_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline1 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.3, random_state=0)

pipeline1.fit(X_train, y_train)

C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\

C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_pred_test = pipeline1.predict(X_test)
y_pred_train = pipeline1.predict(X_train)


In [17]:
# print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))

print(classification_report(y_test, pd.DataFrame(y_pred_test, columns=y_test.columns), target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      6041
               request       0.82      0.42      0.56      1385
                 offer       0.00      0.00      0.00        36
           aid_related       0.74      0.61      0.67      3252
          medical_help       0.54      0.09      0.16       618
      medical_products       0.80      0.11      0.19       408
     search_and_rescue       0.65      0.05      0.10       210
              security       0.00      0.00      0.00       129
              military       0.79      0.10      0.17       238
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.28      0.42       525
                  food       0.84      0.55      0.67       875
               shelter       0.78      0.35      0.48       700
              clothing       0.75      0.05      0.09       130
                 money       0.87      

In [18]:
#Train Accuracy
multi_f1 = multioutput_fscore(y_train,y_pred_train, beta = 1)
overall_accuracy = (y_pred_train == y_train).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 99.30% 

F1 score (custom definition) 99.26%



In [19]:
#Test Accuracy
multi_f1 = multioutput_fscore(y_test,y_pred_test, beta = 1)
overall_accuracy = (y_pred_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.51% 

F1 score (custom definition) 92.91%



C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
# for i in range(36):
#     print(y_test.columns[i], ':')
#     print(classification_report(y_test.iloc[:,i], y_pred_test[:,i], target_names=Y.columns), '...................................................')

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
pipeline1.get_params()
# pipeline.get_params().keys()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001B46C9CB268>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_fea

In [22]:
parameters = {  
#     'clf__estimator__min_samples_split': [2, 4],
#      'clf__estimator__max_features': ['log2', 'auto', 'sqrt', None],
#     'clf__estimator__criterion': ['gini', 'entropy'],
#    'clf__estimator__max_depth': [None, 25, 50, 100, 150, 200],
     'clf__estimator__n_estimators': [10, 20, 40]
    
    
}
cv = GridSearchCV(estimator=pipeline1, param_grid=parameters)
cv.fit(X_train,y_train)

C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [23]:
# Get the prediction values from the grid search cross validator
y_pred_test = cv.predict(X_test)

In [24]:
print(classification_report(y_test, pd.DataFrame(y_pred_test, columns=y_test.columns), target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      6041
               request       0.84      0.46      0.59      1385
                 offer       0.00      0.00      0.00        36
           aid_related       0.76      0.68      0.72      3252
          medical_help       0.62      0.09      0.15       618
      medical_products       0.86      0.07      0.14       408
     search_and_rescue       0.85      0.05      0.10       210
              security       0.33      0.01      0.02       129
              military       0.73      0.07      0.12       238
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.26      0.40       525
                  food       0.86      0.55      0.67       875
               shelter       0.79      0.38      0.51       700
              clothing       0.82      0.11      0.19       130
                 money       0.91      

C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [25]:
multi_f1 = multioutput_fscore(y_test,y_pred_test, beta = 1)
overall_accuracy = (y_pred_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.77% 

F1 score (custom definition) 93.20%



C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
# for i in range(36):
#     print(y_test.columns[i], ':')
#     print(classification_report(y_test.iloc[:,i], y_pred_test[:,i], target_names=Y.columns), '...................................................')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
print(classification_report(y_test, pd.DataFrame(y_pred_test, columns=y_test.columns), target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      6041
               request       0.84      0.46      0.59      1385
                 offer       0.00      0.00      0.00        36
           aid_related       0.76      0.68      0.72      3252
          medical_help       0.62      0.09      0.15       618
      medical_products       0.86      0.07      0.14       408
     search_and_rescue       0.85      0.05      0.10       210
              security       0.33      0.01      0.02       129
              military       0.73      0.07      0.12       238
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.26      0.40       525
                  food       0.86      0.55      0.67       875
               shelter       0.79      0.38      0.51       700
              clothing       0.82      0.11      0.19       130
                 money       0.91      

In [28]:
multi_f1 = multioutput_fscore(y_test,y_pred_test, beta = 1)
overall_accuracy = (y_pred_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.77% 

F1 score (custom definition) 93.20%



In [29]:
# for i in range(36):
#     print(y_test.columns[i], ':')
#     print(classification_report(y_test.iloc[:,i], y_pred_test[:,i], target_names=Y.columns), '...................................................')

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [30]:
#Machine Learning: AdaBoostClassifier()

In [31]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])



In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.3, random_state=0)

pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x000001B46C9CB268>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMM

In [33]:
y_pred_test = pipeline2.predict(X_test)
y_pred_train = pipeline2.predict(X_train)


In [34]:
#Train Accuracy
multi_f1 = multioutput_fscore(y_train,y_pred_train, beta = 1)
overall_accuracy = (y_pred_train == y_train).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 95.05% 

F1 score (custom definition) 93.98%



In [35]:
print(classification_report(y_test, pd.DataFrame(y_pred_test, columns=y_test.columns), target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.80      0.96      0.88      6041
               request       0.74      0.51      0.60      1385
                 offer       0.05      0.03      0.04        36
           aid_related       0.77      0.62      0.68      3252
          medical_help       0.63      0.31      0.42       618
      medical_products       0.66      0.31      0.42       408
     search_and_rescue       0.61      0.19      0.29       210
              security       0.42      0.04      0.07       129
              military       0.60      0.36      0.45       238
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.66      0.71       525
                  food       0.81      0.65      0.72       875
               shelter       0.77      0.53      0.62       700
              clothing       0.72      0.45      0.55       130
                 money       0.57      

C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [36]:
#Test Accuracy
multi_f1 = multioutput_fscore(y_test,y_pred_test, beta = 1)
overall_accuracy = (y_pred_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.80% 

F1 score (custom definition) 93.68%



In [37]:
# for i in range(36):
#     print(y_test.columns[i], ':')
#     print(classification_report(y_test.iloc[:,i], y_pred_test[:,i], target_names=Y.columns), '...................................................')

In [38]:

parameters = {  
#     'clf__estimator__min_samples_split': [2, 4],
#      'clf__estimator__max_features': ['log2', 'auto', 'sqrt', None],
#     'clf__estimator__criterion': ['gini', 'entropy'],
#    'clf__estimator__max_depth': [None, 25, 50, 100, 150, 200],
     'clf__estimator__n_estimators': [10, 20, 40, 50]
    
    
}
cv = GridSearchCV(estimator=pipeline2, param_grid=parameters)
cv.fit(X_train,y_train)

C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [39]:
# Get the prediction values from the grid search cross validator
y_pred_test = cv.predict(X_test)

In [40]:
multi_f1 = multioutput_fscore(y_test,y_pred_test, beta = 1)
overall_accuracy = (y_pred_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.80% 

F1 score (custom definition) 93.68%



In [41]:
print(classification_report(y_test, pd.DataFrame(y_pred_test, columns=y_test.columns), target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.80      0.96      0.88      6041
               request       0.74      0.51      0.60      1385
                 offer       0.05      0.03      0.04        36
           aid_related       0.77      0.62      0.68      3252
          medical_help       0.63      0.31      0.42       618
      medical_products       0.66      0.31      0.42       408
     search_and_rescue       0.61      0.19      0.29       210
              security       0.42      0.04      0.07       129
              military       0.60      0.36      0.45       238
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.66      0.71       525
                  food       0.81      0.65      0.72       875
               shelter       0.77      0.53      0.62       700
              clothing       0.72      0.45      0.55       130
                 money       0.57      

C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\Phongsapon\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [43]:
pickle.dump(cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.